In [42]:
from matplotlib import pyplot as plt
from matplotlib import colors

import numpy as np
import random

import time
from IPython.display import clear_output

grid_rows, grid_columns = 5, 10

# count how many cars in rest area:
cars_ra = 0

# Position y from top to bottom: 0, 1, 2, 3, ...

def create_grid_base(rows, cols):
    grid_base = [[0 for _ in range(cols)] for _ in range(rows)]
    return grid_base

def create_grid_lane(grid_env, pos_y, start, length):
    for x in range(0, length):
        grid_env[pos_y][start + x] = 1
    return grid_env

def create_rest_area(grid_env, pos_y, pos_x):
    grid_env[pos_y][pos_x] = 2
    return grid_env

def create_grid_move(pos_y, pos_x):
    grid_move_0 = create_grid_base(grid_rows, grid_columns)
    grid_move_l = create_grid_base(grid_rows, grid_columns)
    grid_move_s = create_grid_base(grid_rows, grid_columns)
    grid_move_r = create_grid_base(grid_rows, grid_columns)

    try:
        grid_move_0[pos_y][pos_x] = 1
        grid_move_l[pos_y-1][pos_x+1] = 1
        grid_move_s[pos_y+0][pos_x+1] = 1
        grid_move_r[pos_y+1][pos_x+1] = 1
    except: # if agent arrived at destination
        grid_move_0[pos_y][pos_x] = 1
        grid_move_l = grid_move_s = grid_move_r = 0

    return [grid_move_0, grid_move_l, grid_move_s, grid_move_r]

def prop_agent(grid_env, pos_y, pos_x, drowse):
    # if drowse > 5, need to visit rest_area
    #drowse = 1
    grid_pos = create_grid_move(pos_y, pos_x)
    agent = [grid_pos, drowse, grid_env[pos_y][pos_x]] # agent[0] is the position and future movement of the agents
    return agent

def pos_find(grid_move): # for agent position and grid movement
    for i, row_move in enumerate(grid_move):
        for j, val in enumerate(row_move):
            if val == 1:
                pos_y = i
                pos_x = j
                break
    return [pos_y, pos_x]
        
def print_grid(grid):
    for row in grid:
        print(row)

def move_agent(agent, grid_env):
    global cars_ra
    try:
        agent_beneath = agent[2]
        pos_curr = pos_find(agent[0][0])
        pos_next_l = pos_find(agent[0][1]) #[y, x]
        pos_next_s = pos_find(agent[0][2]) #[y, x]
        pos_next_r = pos_find(agent[0][3]) #[y, x]
        drowse = agent[1]
        # take a break
        if agent_beneath == 2:
            if drowse >= 2:
                drowse = drowse - 1
                agent[1] = drowse
            else:
                agent = prop_agent(grid_env, pos_next_r[0], pos_next_r[1], drowse)
                cars_ra = cars_ra - 1
        # move forward, left, right
        elif grid_env[pos_next_l[0]][pos_next_l[1]] == 2 and drowse >= 5:
            agent = prop_agent(grid_env, pos_next_l[0], pos_next_l[1], drowse)
            cars_ra = cars_ra + 1
        elif grid_env[pos_next_s[0]][pos_next_s[1]] == 1:
            drowse = drowse + 0.1
            agent = prop_agent(grid_env, pos_next_s[0], pos_next_s[1], drowse)
        #print_grid(agent[0][0])
        #print()
    except:
        agent = "Arrived"
        print(agent)
    return agent

def grid_agents(agents): # to show the position of the agents
    grid_agent = create_grid_base(grid_rows, grid_columns)
    for agent in agents: # ag stands for agent number ...
        try:
            pos_curr = pos_find(agent[0][0])
            grid_agent[pos_curr[0]][pos_curr[1]] = 1
        except:
            pass
    return grid_agent

def agents_rn_init(grid_env, n, seed=None): # agent random spawn, initial

    if seed is not None:
        random.seed(seed)

    # All coordinates that satisfy the constraint
    eligible = [
        (r, c)
        for r, row in enumerate(grid_env)
        for c, v   in enumerate(row)
        if v == 1
    ]
    print(eligible)
    
    if n > len(eligible):
        raise ValueError(
            f"Only {len(eligible)} eligible spots, but you asked for {n} agents."
        )

    agents = []
    
    for pos_y, pos_x in random.sample(eligible, n):
        print(pos_y, pos_x)
        agent = prop_agent(grid_env, pos_y, pos_x, 1)
        agents.append(agent)

    return agents

In [20]:
# Create the base grid
grid_env = create_grid_base(grid_rows, grid_columns)

# Create highway
grid_env = create_grid_lane(grid_env, 2, 0, grid_columns)
grid_env = create_grid_lane(grid_env, 3, 0, grid_columns)

# Create rest area
grid_env = create_rest_area(grid_env, 1, 3)

# Print environment grid
print_grid(grid_env)
print()

car1 = prop_agent(grid_env, 2, 1, 6)
car2 = prop_agent(grid_env, 2, 0, 2)
car3 = prop_agent(grid_env, 2, 4, 2)
car4 = prop_agent(grid_env, 2, 3, 2)
cars = [car1, car2, car3, car4]
#cars = [car1]

# Create grid for car position
grid_cars = grid_agents(cars)

print_grid(grid_cars)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 2, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 1, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [21]:
# for x in range(len(cars)):
#     cars[x] = move_agent(cars[x], grid_env)

# grid_cars = grid_agents(cars)
# print_grid(grid_cars)
# print()
# print(cars.count("Arrived"), cars_ra)

for i in range(15):
    for x in range(len(cars)):
        cars[x] = move_agent(cars[x], grid_env)
    
    grid_cars = grid_agents(cars)
    print_grid(grid_cars)
    print()
    print("Time elapsed:", i+1)
    print(cars.count("Arrived"), cars_ra)

    # Refresh output
    time.sleep(0.5)
    clear_output(wait=True)

Arrived
Arrived
Arrived
Arrived
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Time elapsed: 15
4 0
